In [1]:
import random
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
from openai import OpenAI
from react_agent import ReactInferenceAgent

In [2]:
class SingleModelOpenAIClient:
    def __init__(self, client, model):
        self._client = client
        self._model = model

    @property
    def chat(self):
        return self

    @property
    def completions(self):
        return self

    def create(self, **kwargs):
        return self._client.chat.completions.create(
            model=self._model,
            **kwargs,
        )

In [3]:
client = SingleModelOpenAIClient(
    client=OpenAI(
        api_key='EMPTY',
        base_url='http://10.239.2.27:18000/v1'
    ),
    model='epoch2epochstep249globalstep749'
)
agent = ReactInferenceAgent(client)

In [4]:
prompt = '''
Based on the provided specifications, select the existing, off-the-shelf circuit from the internal database that best meets the requirements and provide its name. 

The selected circuit does not need to fully satisfy all specifications, but you must always choose the circuit that is the closest overall match among the available database entries, considering reasonable performance trade-offs.

The specifications are:
'''.strip()
print(prompt)

Based on the provided specifications, select the existing, off-the-shelf circuit from the internal database that best meets the requirements and provide its name. 

The selected circuit does not need to fully satisfy all specifications, but you must always choose the circuit that is the closest overall match among the available database entries, considering reasonable performance trade-offs.

The specifications are:


In [5]:
# df = pd.read_csv('/nas_train/app.e0030265/review_datas/COT_data_1127.csv')
df = pd.read_csv('/nas_train/app.e0030265/review_datas/new3000_0109.csv')
tr = df.sample(n=2000, random_state=42)
df = df[~df.index.isin(tr.index)]
df = df.sample(n=1000, random_state=42)
df

,IP,SPEC
1597,FD_CAS_OTA,Please design a OTA circuit:\nCircuit type: Ty...
2828,FD_CAS_OTA,Please design a OTA circuit:\nCircuit type: Ty...
2217,CM_OTA,Please design a OTA circuit:\nCircuit type: Ty...
2933,CM_OTA,Please design a OTA circuit:\nCircuit type: Ty...
1246,FT_OTA,Please design a OTA circuit:\nCircuit type: Ty...
...,...,...
200,FD_CAS_OTA,Please design a OTA circuit:\nCircuit type: Ty...
333,FD_CAS_OTA,Please design a OTA circuit:\nCircuit type: Ty...
828,FT_OTA,Please design a OTA circuit:\nCircuit type: Ty...
2584,CM_OTA,Please design a OTA circuit:\nCircuit type: Ty...


In [6]:
result = []

for i in tqdm(df.index):
    query = df.loc[i, 'SPEC'].strip()
    query = query.splitlines()[1:]
#     random.shuffle(query)
    query = '\n'.join(query)
    query = f'{prompt}\n{query}'
    target = df.loc[i, 'IP'].strip()
    result.append({
        'question': query,
        'answer': target
    })

df = pd.DataFrame(result)
df

  0%|          | 0/1000 [00:00<?, ?it/s]

,question,answer
0,"Based on the provided specifications, select t...",FD_CAS_OTA
1,"Based on the provided specifications, select t...",FD_CAS_OTA
2,"Based on the provided specifications, select t...",CM_OTA
3,"Based on the provided specifications, select t...",CM_OTA
4,"Based on the provided specifications, select t...",FT_OTA
...,...,...
995,"Based on the provided specifications, select t...",FD_CAS_OTA
996,"Based on the provided specifications, select t...",FD_CAS_OTA
997,"Based on the provided specifications, select t...",FT_OTA
998,"Based on the provided specifications, select t...",CM_OTA


In [7]:
tp = df.sample(n=1)
question = tp.iloc[0]['question']
answer = tp.iloc[0]['answer']
print(question)
print(answer)

Based on the provided specifications, select the existing, off-the-shelf circuit from the internal database that best meets the requirements and provide its name. 

The selected circuit does not need to fully satisfy all specifications, but you must always choose the circuit that is the closest overall match among the available database entries, considering reasonable performance trade-offs.

The specifications are:
Circuit type: Type : OTA
Supply voltage: Vdd : = 2.3V
closed-loop gain: Av : >=34dB
Gain-bandwidth product: GBW : >=1791MHz
Phase margin: PM : >=71degree
Input offset voltage: Vos : <=6mV
Output noise PSD: PSD_Vout : <=20nV/√Hz
Power Supply Rejection ratio: PSRR : >=30dB
Common Mode Rejection ratio: CMRR : >=54dB
Slew Rate: SR : >=140V/us
Settling Time: Tsettle : <=14ns
FT_OTA


In [8]:
content, messages = agent.run(question)

In [9]:
print(content)

<think>
Okay, let's see. The user is looking for an OTA circuit that meets specific specifications. The key parameters are Vdd of 2.3V, closed-loop gain (Av) >=34dB, GBW >=1791MHz, PM >=71 degrees, Vos <=6mV, PSD_Vout <=20nV/√Hz, PSRR >=30dB, CMRR >=54dB, and SR >=140V/us. 

First, I need to check the available circuits. From the previous tool calls, the relevant OTAs are CM_OTA, FD_CAS_OTA, FT_OTA, and TS_MILLER_OTA. Let's go through each one.

Starting with CM_OTA: The Vdd is specified as 1.2V to 1.8V, but the user needs 2.3V. Wait, the min_value for Vdd is 1.2V, max is None, but the description says "Low" supply voltage. So maybe it's not suitable for 2.3V. The Av is 40-60dB, which meets the 34dB requirement. GBW is 300-1000MHz, but the user needs 1791MHz, so this is too low. PM is 60-80 degrees, which is good. Vos is 4-8mV, which is within the 6mV limit. PSD_Vout is 25-50nV/√Hz, but the user wants <=20, so this is over. PSRR is 40-60dB, which meets 30dB. CMRR is 50-70dB, which meet

In [10]:
# messages = [m for m in messages if m['role'] == 'assistant']
# print(len(messages))
# for m in messages:
#     print('-' * 100)
#     print(m)

In [11]:
messages

[{'role': 'system',
  'content': 'You are a helpful assistant engaged in a multi-turn conversation with a user.\nYour task is to answer the user\'s question step by step, using tools only when necessary.\n\nAvailable tools are defined within `<tools></tools>` tags:\n<tools>\n{"type": "function", "function": {"name": "get_all_circuit_summaries", "description": "Retrieves a high-level catalog of all available circuit IPs, including their official names and brief functional descriptions.", "parameters": {"type": "object", "properties": {}, "required": []}}}\n{"type": "function", "function": {"name": "get_circuit_specs_by_name", "description": "Retrieves the detailed technical reference specifications for specific circuits identified by their names.", "parameters": {"type": "object", "properties": {"names": {"type": "array", "items": {"type": "string"}, "description": "List of circuit names to query."}}, "required": ["names"]}}}\n</tools>\n\nInteraction Rules:\n- You may call at most one t